In [ ]:
import anndata as ad
import pandas as pd
import ktplotspy as kpy
import matplotlib.pyplot as plt
import os

In [ ]:
# Load CellphoneDB results
adata_CM = ad.read_h5ad(r"C:\Users\juanp\Desktop\CellphoneDB_CM_Other\CM_counts.h5ad")
means_CM = pd.read_csv(r"C:\Users\juanp\Desktop\CellphoneDB_Results_CM_Other\CM\statistical_analysis_means_04_07_2025_191005.txt", sep="\t")
pvals_CM = pd.read_csv(r"C:\Users\juanp\Desktop\CellphoneDB_Results_CM_Other\CM\statistical_analysis_pvalues_04_07_2025_191005.txt", sep="\t")
decon_CM = pd.read_csv(r"C:\Users\juanp\Desktop\CellphoneDB_Results_CM_Other\CM\statistical_analysis_deconvoluted_04_07_2025_191005.txt", sep="\t")
interaction_scores_CM = pd.read_csv(r"C:\Users\juanp\Desktop\CellphoneDB_Results_CM_Other\CM\statistical_analysis_interaction_scores_04_07_2025_191005.txt", sep="\t")
significant_means_CM = pd.read_csv(r"C:\Users\juanp\Desktop\CellphoneDB_Results_CM_Other\CM\statistical_analysis_significant_means_04_07_2025_191005.txt", sep="\t")

adata_CMEC = ad.read_h5ad(r"C:\Users\juanp\Desktop\CellphoneDB_CM_Other\CMEC_counts.h5ad")
means_CMEC = pd.read_csv(r"C:\Users\juanp\Desktop\CellphoneDB_Results_CM_Other\CMEC\statistical_analysis_means_04_07_2025_191441.txt", sep="\t")
pvals_CMEC = pd.read_csv(r"C:\Users\juanp\Desktop\CellphoneDB_Results_CM_Other\CMEC\statistical_analysis_pvalues_04_07_2025_191441.txt", sep="\t")
decon_CMEC = pd.read_csv(r"C:\Users\juanp\Desktop\CellphoneDB_Results_CM_Other\CMEC\statistical_analysis_deconvoluted_04_07_2025_191441.txt", sep="\t")
interaction_scores_CMEC = pd.read_csv(r"C:\Users\juanp\Desktop\CellphoneDB_Results_CM_Other\CMEC\statistical_analysis_interaction_scores_04_07_2025_191441.txt", sep="\t")
significant_means_CMEC = pd.read_csv(r"C:\Users\juanp\Desktop\CellphoneDB_Results_CM_Other\CMEC\statistical_analysis_significant_means_04_07_2025_191441.txt", sep="\t")

output_dir = r"C:\Users\juanp\Desktop\CellphoneDB visualization CM Other"
os.makedirs(output_dir, exist_ok=True)

### Counting how many interactions happened in each condition
The CM condition has almost as many interactions between different cell types as the CMEC condition. But when we look at the significant interactions EC -> CM or CM -> EC it has way less. 

In [ ]:
# what columns to look at (all columns with |)
cols_CMEC = [col for col in significant_means_CMEC.columns if '|' in col]
cols_CM = [col for col in significant_means_CM.columns if '|' in col]

#Count how many rows have a value
count_CMEC = significant_means_CMEC[cols_CMEC].notna().sum().sum()
count_CM = significant_means_CM[cols_CM].notna().sum().sum()

In [ ]:
print (count_CM)
print (count_CMEC)

In [ ]:
# Plotting count of significant interactions in ALL cells
barplot_path = os.path.join(output_dir, "all_significant_interactions_barplot.png")
conditions = ["CM", "CM-EC"]
all_interactions = [count_CM, count_CMEC]
plt.figure(figsize=(6, 4))
plt.bar(conditions, all_interactions, color=['skyblue', 'salmon'])
plt.ylabel("Significant Interactions Count")
plt.title("Significant Interactions (All Cells)")
plt.savefig(barplot_path, dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Count significant interactions in ec -> cm and cm -> ec
# what columns to look at
other_cm_col = "Other|Cardiomyocyte"
cm_other_col = "Cardiomyocyte|Other"
cm_cm_col = "Cardiomyocyte|Cardiomyocyte"
other_other_col = "Other|Other"

# Count how many rows have a value

other_cm_count_CM = significant_means_CM[other_cm_col].notna().sum()
cm_other_count_CM = significant_means_CM[cm_other_col].notna().sum()
cm_cm_count_CM = significant_means_CM[cm_cm_col].notna().sum()
other_other_count_CM = significant_means_CM[other_other_col].notna().sum()

other_cm_count_CMEC = significant_means_CMEC[other_cm_col].notna().sum()
cm_other_count_CMEC = significant_means_CMEC[cm_other_col].notna().sum()
cm_cm_count_CMEC = significant_means_CMEC[cm_cm_col].notna().sum()
other_other_count_CMEC = significant_means_CMEC[other_other_col].notna().sum()


In [ ]:
print (other_cm_count_CM)
print (cm_cm_count_CM)
print (cm_other_count_CM)
print (other_other_count_CM)

print (other_cm_count_CMEC)
print (cm_cm_count_CMEC)
print (cm_other_count_CMEC)
print (other_other_count_CMEC)

In [ ]:
# Significant interactions from Endothelial Cell to Cardiomyocyte
barplot_path2 = os.path.join(output_dir, "othertocm_significant_interactions_barplot.png")
other_to_cm = [other_cm_count_CM, other_cm_count_CMEC]
plt.figure(figsize=(6, 4))
plt.bar(conditions, other_to_cm, color=['skyblue', 'salmon'])
plt.ylabel("Significant Interactions Count")
plt.title("Significant Interactions (Other to CM)")
plt.savefig(barplot_path2, dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Significant interactions from Cardiomyocyte to Endothelial Cell
barplot_path3 = os.path.join(output_dir, "Cmtoother_significant_interactions_barplot.png")
cm_to_other = [cm_other_count_CM, cm_other_count_CMEC]
plt.figure(figsize=(6, 4))
plt.bar(conditions, cm_to_other, color=['skyblue', 'salmon'])
plt.ylabel("Significant Interactions Count")
plt.title("Significant Interactions (CM to Other)")
plt.savefig(barplot_path3, dpi=300, bbox_inches="tight")
plt.show()

### What are the unique significant interactions between EC and CM in the CMEC?
I am looking at the significant means file and comparing the interacting pairs in the CM vs CMEC and seeing what pairs we have in the CMEC that are not in CM. 

In [ ]:
# Get significant EC → CM interactions in both datasets
other_cm_CMEC = significant_means_CMEC[significant_means_CMEC[other_cm_col].notna()]
other_cm_CM = significant_means_CM[significant_means_CM[other_cm_col].notna()]

In [ ]:
# Get significant CM → EC interactions in both datasets
cm_other_CMEC = significant_means_CMEC[significant_means_CMEC[cm_other_col].notna()]
cm_other_CM = significant_means_CM[significant_means_CM[cm_other_col].notna()]

In [ ]:
# Find shared interacting pairs between CMEC and CM (ec to cm)
other_cm_shared_pairs = set(other_cm_CM["interacting_pair"]) & set(other_cm_CMEC["interacting_pair"])

In [ ]:
# Find shared interacting pairs between CMEC and CM (cm to ec)
cm_other_shared_pairs = set(cm_other_CM["interacting_pair"]) & set(cm_other_CMEC["interacting_pair"])

In [ ]:
# Keep only interactions unique to CMEC
unique_other_cm_CMEC = other_cm_CMEC[~other_cm_CMEC["interacting_pair"].isin(other_cm_shared_pairs)]

In [ ]:
# Keep only interactions unique to CMEC
unique_cm_other_CMEC = cm_other_CMEC[~cm_other_CMEC["interacting_pair"].isin(cm_other_shared_pairs)]

In [ ]:
# Get top 10 by mean expression
top10_unique_other_cm = unique_other_cm_CMEC[["interacting_pair", "classification", other_cm_col]]\
    .sort_values(by=other_cm_col, ascending=False).head(10)

top10_unique_other_cm

In [ ]:
# Get top 10 by mean expression
top10_unique_cm_other = unique_cm_other_CMEC[["interacting_pair", "classification", cm_other_col]]\
    .sort_values(by=cm_other_col, ascending=False).head(10)

top10_unique_cm_other

### Heatmaps: Plots the significant interactions between the different cells

In [ ]:
# Heatmaps for all cells in CM
save_path_heatmap1 = os.path.join(output_dir, "heatmap_CM_all_cells.png")
kpy.plot_cpdb_heatmap(pvals=pvals_CM, figsize=(5, 5), title=" Significant interactions - CM")
plt.savefig(save_path_heatmap1, dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
tablita = kpy.plot_cpdb_heatmap(pvals=pvals_CM, figsize=(5, 5), title=" Significant interactions - CM", symmetrical=False, return_tables=True)
interaction_matrix2 = tablita["count_network"]
interaction_matrix2


In [ ]:
tablita2 = kpy.plot_cpdb_heatmap(pvals=pvals_CMEC, figsize=(5, 5), title=" Significant interactions - CM", symmetrical=False, return_tables=True)
interaction_matrix3 = tablita2["count_network"]
interaction_matrix3


In [ ]:
kpy.plot_cpdb_heatmap(pvals=pvals_CM, figsize=(5, 5), title=" Significant interactions - CM", symmetrical=False)

In [ ]:
kpy.plot_cpdb_heatmap(pvals=pvals_CMEC, figsize=(5, 5), title=" Significant interactions - CM-EC", symmetrical=False)

In [ ]:
# Heatmaps for all cells in CMEC
save_path_heatmap2 = os.path.join(output_dir, "heatmap_CMEC_all_cells.png")
kpy.plot_cpdb_heatmap(pvals=pvals_CMEC, figsize=(5, 5), title="Significant interactions - CMEC")
plt.savefig(save_path_heatmap2, dpi=300, bbox_inches="tight")
plt.show()

### Dot plots

In [ ]:
p_VEGF = kpy.plot_cpdb(
    adata=adata_CMEC,
    cell_type1="Cardiomyocyte",
    cell_type2=".",
    means=means_CMEC,
    pvals=pvals_CMEC,
    celltype_key="cell_type",
    genes=["VEGF"],
    figsize=(12, 5),
    title="VEGF related signaling interactions"
)
dot_path1 = r"C:\Users\juanp\Desktop\CellphoneDB visualization CM Other\vegf_dotplot_CMEC.png"
p_VEGF.save (dot_path1, dpi = 300)
p_VEGF

In [ ]:
p_NRG = kpy.plot_cpdb(
    adata=adata_CMEC,
    cell_type1="Cardiomyocyte",
    cell_type2=".",
    means=means_CMEC,
    pvals=pvals_CMEC,
    celltype_key="cell_type",
    genes=["NRG"],
    figsize=(12, 5),
    title="NRG related signaling interactions"
)
dot_path2 = r"C:\Users\juanp\Desktop\CellphoneDB visualization CM Other\nrg_dotplot_CMEC.png"
p_NRG.save(dot_path2, dpi = 300)
p_NRG

In [ ]:
p_EDN1 = kpy.plot_cpdb(
    adata=adata_CMEC,
    cell_type1="Cardiomyocyte",
    cell_type2=".",
    means=means_CMEC,
    pvals=pvals_CMEC,
    celltype_key="cell_type",
    genes=["EDN1"],
    figsize=(12, 5),
    title="EDN1 related signaling interactions"
)
dot_path3 = r"C:\Users\juanp\Desktop\CellphoneDB visualization CM Other\edn1_dotplot_CMEC.png"
p_EDN1.save(dot_path3, dpi = 300)
p_EDN1

In [ ]:
p_FGF = kpy.plot_cpdb(
    adata=adata_CMEC,
    cell_type1="Cardiomyocyte",
    cell_type2=".",
    means=means_CMEC,
    pvals=pvals_CMEC,
    celltype_key="cell_type",
    genes=["FGF"],
    figsize=(12, 5),
    title="FGF related signaling interactions"
)
dot_path4 = r"C:\Users\juanp\Desktop\CellphoneDB visualization CM Other\fgf_dotplot_CMEC.png"
p_FGF.save(dot_path4, dpi = 300)
p_FGF

In [ ]:
p_COL = kpy.plot_cpdb(
    adata=adata_CMEC,
    cell_type1="Cardiomyocyte",
    cell_type2=".",
    means=means_CMEC,
    pvals=pvals_CMEC,
    celltype_key="cell_type",
    genes=["COL"],
    figsize=(12, 5),
    title="COL related signaling interactions"
)
dot_path5 = r"C:\Users\juanp\Desktop\CellphoneDB visualization\col_dotplot_CMEC.png"
p_COL.save(dot_path5, dpi = 300)
p_COL

In [ ]:
p_FN = kpy.plot_cpdb(
    adata=adata_CMEC,
    cell_type1="Cardiomyocyte",
    cell_type2=".",
    means=means_CMEC,
    pvals=pvals_CMEC,
    celltype_key="cell_type",
    genes=["FN"],
    figsize=(12, 5),
    title="FN related signaling interactions"
)
dot_path6 = r"C:\Users\juanp\Desktop\CellphoneDB visualization CM Other\fn_dotplot_CMEC.png"
p_FN.save(dot_path6, dpi = 300)
p_FN

In [ ]:
p_FN = kpy.plot_cpdb(
    adata=adata_CMEC,
    cell_type1="Cardiomyocyte",
    cell_type2=".",
    means=means_CMEC,
    pvals=pvals_CMEC,
    celltype_key="cell_type",
    genes=["FN"],
    figsize=(12, 5),
    title="FN related signaling interactions"
)
dot_path6 = r"C:\Users\juanp\Desktop\CellphoneDB visualization\fn_dotplot_CMEC.png"
p_FN.save(dot_path6, dpi = 300)
p_FN

### Chord diagrams

In [ ]:
pc_VEGF = kpy.plot_cpdb_chord(
    adata=adata_CMEC,
    cell_type1="Cardiomyocyte",
    cell_type2=".",
    means=means_CMEC,
    pvals=pvals_CMEC,
    deconvoluted=decon_CMEC,
    celltype_key="cell_type",
    interaction=["VEGF"],
    link_kwargs={"direction": 1, "allow_twist": True, "r1": 95, "r2": 90},
    sector_text_kwargs={"color": "black", "size": 12, "r": 105, "adjust_rotation": True},
    legend_kwargs={"loc": "center", "bbox_to_anchor": (1, 1), "fontsize": 8},
    link_offset=1,
)
chord_path1 = r"C:\Users\juanp\Desktop\CellphoneDB visualization CM Other\VEGF_chord_CMEC.png"
pc_VEGF.savefig(chord_path1, dpi = 300)
pc_VEGF

In [ ]:
pc_NRG = kpy.plot_cpdb_chord(
    adata=adata_CMEC,
    cell_type1="Cardiomyocyte",
    cell_type2=".",
    means=means_CMEC,
    pvals=pvals_CMEC,
    deconvoluted=decon_CMEC,
    celltype_key="cell_type",
    interaction=["NRG"],
    link_kwargs={"direction": 1, "allow_twist": True, "r1": 95, "r2": 90},
    sector_text_kwargs={"color": "black", "size": 12, "r": 105, "adjust_rotation": True},
    legend_kwargs={"loc": "center", "bbox_to_anchor": (1, 1), "fontsize": 8},
    link_offset=1,
)
chord_path2 = r"C:\Users\juanp\Desktop\CellphoneDB visualization CM Other\NRG_chord_CMEC.png"
pc_NRG.savefig(chord_path2, dpi = 300)
pc_NRG

In [ ]:
pc_EDN = kpy.plot_cpdb_chord(
    adata=adata_CMEC,
    cell_type1="Cardiomyocyte",
    cell_type2=".",
    means=means_CMEC,
    pvals=pvals_CMEC,
    deconvoluted=decon_CMEC,
    celltype_key="cell_type",
    interaction=["EDN"],
    link_kwargs={"direction": 1, "allow_twist": True, "r1": 95, "r2": 90},
    sector_text_kwargs={"color": "black", "size": 12, "r": 105, "adjust_rotation": True},
    legend_kwargs={"loc": "center", "bbox_to_anchor": (1, 1), "fontsize": 8},
    link_offset=1,
)
chord_path3 = r"C:\Users\juanp\Desktop\CellphoneDB visualization CM Other\EDN_chord_CMEC.png"
pc_EDN.savefig(chord_path3, dpi = 300)
pc_EDN

In [ ]:
pc_COL = kpy.plot_cpdb_chord(
    adata=adata_CMEC,
    cell_type1="Cardiomyocyte",
    cell_type2=".",
    means=means_CMEC,
    pvals=pvals_CMEC,
    deconvoluted=decon_CMEC,
    celltype_key="cell_type",
    interaction=["COL"],
    link_kwargs={"direction": 1, "allow_twist": True, "r1": 95, "r2": 90},
    sector_text_kwargs={"color": "black", "size": 12, "r": 105, "adjust_rotation": True},
    legend_kwargs={"loc": "center", "bbox_to_anchor": (1, 1), "fontsize": 8},
    link_offset=1,
)
chord_path4 = r"C:\Users\juanp\Desktop\CellphoneDB visualization CM Other\COL_chord_CMEC.png"
pc_COL.savefig(chord_path4, dpi = 300)
pc_COL

In [ ]:
import scanpy as sc

sc.pl.violin(adata_CMEC, ["n_genes"], groupby="cell_type", rotation=90)

### Most important classifications: 
I am looking at the significant means file, the ec to cm column. Categorize by classification. Sum the mean expression of all interacting pairs in that category. 


In [ ]:
# Filter rows that have significant EC->CM interaction
significant_other_cm_CM = significant_means_CM[significant_means_CM[other_cm_col].notna()]
significant_other_cm_CMEC = significant_means_CMEC[significant_means_CMEC[other_cm_col].notna()]

# Filter rows that have significant CM->EC interaction
significant_cm_other_CM = significant_means_CM[significant_means_CM[cm_other_col].notna()]
significant_cm_other_CMEC = significant_means_CMEC[significant_means_CMEC[cm_other_col].notna()]

In [ ]:
# Group by classification and sum the expression values
#EC to CM - cm
cumulative_expression_other_CM_CM = (
    significant_other_cm_CM
    .groupby("classification")[other_cm_col]
    .sum()
    .sort_values(ascending=False)
    .reset_index()
)
#EC to CM - cmec
cumulative_expression_other_CM_CMEC = (
    significant_other_cm_CMEC
    .groupby("classification")[other_cm_col]
    .sum()
    .sort_values(ascending=False)
    .reset_index()
)
#CM to EC - cm
cumulative_expression_CM_other_CM = (
    significant_cm_other_CM
    .groupby("classification")[cm_other_col]
    .sum()
    .sort_values(ascending=False)
    .reset_index()
)
#CM to EC - cmec
cumulative_expression_CM_other_CMEC = (
    significant_cm_other_CMEC
    .groupby("classification")[cm_other_col]
    .sum()
    .sort_values(ascending=False)
    .reset_index()
)


In [ ]:
# Show top classifications
cumulative_expression_other_CM_CM.head(15)

In [ ]:
cumulative_expression_other_CM_CMEC.head(15)

In [ ]:
cumulative_expression_CM_other_CM.head(15)

In [ ]:
cumulative_expression_CM_other_CMEC.head(15)